In [5]:
# lunarlander_neat.py

import gymnasium as gym
import neat
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = 0.0

        for _ in range(TRAIN_EPISODES):
            obs, _ = env.reset()
            done = False
            total_reward = 0

            while not done:
                output = net.activate(obs)
                action = np.argmax(output)
                obs, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                total_reward += reward

            genome.fitness += total_reward


def test_winner(winner, config, episodes=5):
    env_test = gym.make("LunarLander-v3", render_mode="human")
    net = neat.nn.FeedForwardNetwork.create(winner, config)

    for i in range(episodes):
        obs, _ = env_test.reset()
        done = False
        total_reward = 0
        print(f"\n🚀 Landing {i + 1}")

        while not done:
            output = net.activate(obs)
            action = np.argmax(output)
            obs, reward, terminated, truncated, _ = env_test.step(action)
            done = terminated or truncated
            total_reward += reward

        print(f"✅ Total reward: {total_reward:.2f}")

    env_test.close()


def plot_stats(stats):
    generations = range(len(stats.most_fit_genomes))
    best_fitness = [g.fitness for g in stats.most_fit_genomes]
    avg_fitness = stats.get_fitness_mean()

    plt.figure(figsize=(10, 5))
    plt.plot(generations, best_fitness, label="Best Fitness")
    plt.plot(generations, avg_fitness, label="Average Fitness")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.title("NEAT Training Progress - LunarLander")
    plt.legend()
    plt.grid()
    plt.show()


def run_neat(config_path, skip_training=False):
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path
    )

    model_path = "lunarlander_winner.pkl"

    if skip_training and os.path.exists(model_path):
        print("🔁 Loading saved model...")
        with open(model_path, "rb") as f:
            winner = pickle.load(f)
        test_winner(winner, config)
        return

    print("🚀 Starting training...")
    population = neat.Population(config)
    stats = neat.StatisticsReporter()
    population.add_reporter(neat.StdOutReporter(True))
    population.add_reporter(stats)

    winner = population.run(eval_genomes, GENERATIONS)

    with open(model_path, "wb") as f:
        pickle.dump(winner, f)

    print("✅ Training complete. Model saved.")
    plot_stats(stats)
    test_winner(winner, config)

TRAIN_EPISODES = 10
GENERATIONS = 50

# Create training env globally
env = gym.make("LunarLander-v3")

# Run it (choose mode)
run_neat("config-feedforward_lunarLander", skip_training=True)


🔁 Loading saved model...

🚀 Landing 1
✅ Total reward: 222.66

🚀 Landing 2
✅ Total reward: 283.85

🚀 Landing 3
✅ Total reward: 202.92

🚀 Landing 4
✅ Total reward: 270.18

🚀 Landing 5
✅ Total reward: 260.12
